In [5]:
import re
import pandas as pd
from tqdm import tqdm

## finding the unique words


In [4]:
with open('big.txt','r') as fd:
    lines = fd.readlines()
    words=[]
    for line in lines:
        words+=re.findall('\w+',line.lower())
vocab= set(words)
print(len(vocab))

32198


## find the probabilty distribution

In [6]:
word_prob={}
for word in tqdm(vocab):
    word_prob[word]=float(words.count(word)/len(words))

100%|████████████████████████████████████████████████████████████████████████████| 32198/32198 [14:05<00:00, 38.10it/s]


# Text preprocessing 
## splitting 


In [7]:
def spliting(word):
    parts=[]
    for i in range(len(word)+1):
        parts+= [(word[:i],word[i:])]
    return parts


In [8]:
spliting('spartan')

[('', 'spartan'),
 ('s', 'partan'),
 ('sp', 'artan'),
 ('spa', 'rtan'),
 ('spar', 'tan'),
 ('spart', 'an'),
 ('sparta', 'n'),
 ('spartan', '')]

## Delete 


In [9]:
def delete(word):
    output=[]
    for l,r in spliting(word):
        output.append(l+r[1:])
    return output
delete('spartan')

['partan',
 'sartan',
 'sprtan',
 'spatan',
 'sparan',
 'spartn',
 'sparta',
 'spartan']

## swaping

In [10]:
def swap(word):
    output=[]
    for l,r in spliting(word):
        if len(r)>1:
            output.append(l+r[1]+r[0]+r[2:])
    return output
swap('spartan')

['psartan', 'saprtan', 'spratan', 'spatran', 'sparatn', 'spartna']

## Replace


In [15]:
def replace(word):
    output=[]
    characters= 'abcdefghijklmnopqrstuvwxyz'
    for l,r in spliting(word):
        for char in characters:
            output.append(l+char+r[1:])
    return output
len(replace('spartan'))

208

## insert


In [17]:
def insert(word):
    output=[]
    characters= 'abcdefghijklmnopqrstuvwxyz'
    for l,r in spliting(word):
        for char in characters:
            output.append(l+char+r)
    return output
len(insert('spartan'))

208

# Finding the prediction 
## combining the possible words


In [20]:
def edit(word):
    return set(insert(word)+delete(word)+replace(word)+swap(word))
len(edit('spartan'))

390

## predicted the word


In [26]:
def spelingcheck(word,count=5):
    output=[]
    suggestedword=edit(word)
    for wrd in suggestedword:
        if wrd in word_prob.keys():
            output.append([wrd,word_prob[wrd]])
    return list(pd.DataFrame(output, columns=['word','prob']).sort_values(by='prob',ascending=False).head(count)['word'].values)
spelingcheck('the')

['the', 'he', 'she', 'they', 'them']

## predict the word at level two 

In [32]:
def spelingcheck_2(word,count=5):
    output=[]
    suggestedword=edit(word) # level one 
    for e1 in edit(word):
        suggestedword += edit(e1) # level 2
    suggestedword = list(set(suggestedword))
    for wrd in suggestedword:
        if wrd in word_prob.keys():
            output.append([wrd,word_prob[wrd]])
    return list(pd.DataFrame(output, columns=['word','prob']).sort_values(by='prob',ascending=False).head(count)['word'].values)
